# Python Data Analysis

The **target** value or **label** is the value that we would like to predict using the other variables.

In [28]:
import pandas as pd
import numpy as np
df = pd.read_csv("assets/sales.csv")
df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Tuvalu,Baby Food,Offline,H,5/28/2010,669165933,6/27/2010,9925,255.28,159.42,2533654.00,1582243.50,951410.50
1,Central America and the Caribbean,Grenada,Cereal,Online,C,8/22/2012,963881480,9/15/2012,2804,205.70,117.11,576782.80,328376.44,248406.36
2,Europe,Russia,Office Supplies,Offline,L,5/2/2014,341417157,5/8/2014,1779,651.21,524.96,1158502.59,933903.84,224598.75
3,Sub-Saharan Africa,Sao Tome and Principe,Fruits,Online,C,6/20/2014,514321792,7/5/2014,8102,9.33,6.92,75591.66,56065.84,19525.82
4,Sub-Saharan Africa,Rwanda,Office Supplies,Offline,L,2/1/2013,115456712,2/6/2013,5062,651.21,524.96,3296425.02,2657347.52,639077.50


## Pre-Processing

After the data has been loaded, we then apply an assortment of techniques to prepare the raw data for analysis. This includes: dealing with missing values, data formatting and normalization/scaling.

#### Dealing With Missing Values

Missing values are common. First, check with the collection source to see whether the missing data can be obtained or estimated. If not, there are several options:

* Replace the missing value:
    * With the average (of similar data points)
    * If it is a categorical variable, replace with the mode
    * Use another estimation technique
* Or, you may choose to remove the missing data:
    * Either by dropping the data entry (row): ```axis = 0```
    * Or by dropping the entire variable (column): ```axis = 1```

In [29]:
df.dropna(subset = ["Sales Channel"], axis = 0) # Returns a new modified dataframe
df.dropna(subset = ["Sales Channel"], axis = 0, inplace = True) # Modifies dataframe in-place

In [32]:
avg = df["Units Sold"].mean()
df["Units Sold"] = df["Units Sold"].replace(np.nan, avg) # Replace NaN with mean value

In [33]:
df.head()

,Region,Country,Item Type,Sales Channel,Order Priority,Order Date,Order ID,Ship Date,Units Sold,Unit Price,Unit Cost,Total Revenue,Total Cost,Total Profit
0,Australia and Oceania,Tuvalu,Baby Food,Offline,H,5/28/2010,669165933,6/27/2010,9925,255.28,159.42,2533654.00,1582243.50,951410.50
1,Central America and the Caribbean,Grenada,Cereal,Online,C,8/22/2012,963881480,9/15/2012,2804,205.70,117.11,576782.80,328376.44,248406.36
2,Europe,Russia,Office Supplies,Offline,L,5/2/2014,341417157,5/8/2014,1779,651.21,524.96,1158502.59,933903.84,224598.75
3,Sub-Saharan Africa,Sao Tome and Principe,Fruits,Online,C,6/20/2014,514321792,7/5/2014,8102,9.33,6.92,75591.66,56065.84,19525.82
4,Sub-Saharan Africa,Rwanda,Office Supplies,Offline,L,2/1/2013,115456712,2/6/2013,5062,651.21,524.96,3296425.02,2657347.52,639077.50
